In [245]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import pairwise_distances
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from scipy.spatial.distance import cosine, correlation
from surprise import Reader, Dataset, SVD, NormalPredictor, BaselineOnly, KNNBasic, NMF
from surprise.model_selection import cross_validate, KFold ,GridSearchCV , RandomizedSearchCV
from surprise import Dataset

In [246]:
df=pd.read_csv('ratings.csv', usecols = ['user_id','movie_id','rating'])
df

,user_id,movie_id,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [247]:
user_df=pd.DataFrame(data=df['user_id'].unique(), columns=["user_id"])
user_df

,user_id
0,1
1,2
2,3
3,4
4,5
...,...
605,606
606,607
607,608
608,609


In [248]:
pivot_table_user = pd.pivot_table(data=df,values='rating',index='user_id',columns='movie_id')
pivot_table_user = pivot_table_user.fillna(0)
#for i in pivot_table_user:
    #print(pivot_table_user[600][i])
pivot_table_user

movie_id,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
user_id,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [249]:
df2=pd.read_csv('sim2.csv')
df2

,user1,sim,user2
0,1,0.603112,19
1,1,0.651583,7
2,1,0.734835,11
3,1,0.653312,14
4,1,0.585003,4
...,...,...,...
146741,610,0.696254,487
146742,610,0.615739,294
146743,610,0.633902,528
146744,610,0.750000,340


In [250]:
sim = pd.pivot_table(data=df2,values='sim',index='user1',columns='user2')
sim[5][26]

0.8418861169915809

In [251]:
for i in range(1,175):
    sim[i][i]=1
for i in range(176,611):
    sim[i][i]=1
sim
sim=sim.fillna(0)
a=sim[5].sort_values(ascending = False)
sayac=0
for i in a:
    print(a.index[sayac]," ",i)
    sayac+=1
    if sayac==50:
        break

5   1.0
26   0.8418861169915809
596   0.8266561886796159
254   0.8232233047033631
145   0.8232233047033631
609   0.8063508326896291
332   0.8056796030606497
534   0.8005689911956336
514   0.7984435562925363
102   0.7958758547680684
182   0.7952967780153202
199   0.7943804197925064
166   0.7862873964337224
357   0.7834936490538904
599   0.7834936490538904
275   0.7807354951732427
142   0.7795207240779507
590   0.7790291308792039
134   0.7763932022500211
577   0.7738664915666773
370   0.7729262234426881
448   0.7712866802989042
565   0.7705842661294382
288   0.7690443392307101
290   0.7685449750568621
425   0.7676699056788537
436   0.7668737979399216
169   0.7648650142185099
103   0.7632170266455203
282   0.7616343526886019
298   0.7586731752091442
202   0.7563301413797758
347   0.7563301413797758
33   0.7560249817628667
72   0.7531447792733562
144   0.7529941199851616
17   0.75
444   0.75
587   0.75
411   0.7457261861421961
446   0.745237667317466
404   0.7443813209272274
233   0.743826

In [252]:
a.index[0]

5

In [253]:
sim.index[0]

1

In [254]:
sim[5]
sim[5].sort_values(ascending = False)

user1
5      1.000000
26     0.841886
596    0.826656
254    0.823223
145    0.823223
         ...   
311    0.000000
312    0.000000
315    0.000000
316    0.000000
306    0.000000
Name: 5, Length: 609, dtype: float64

In [255]:
sim[2][596]

0.8357178148318073

In [236]:
def rec_user1(user_id):
    temp_table = pd.DataFrame(columns = user_df.columns)
    #us = sim[user_id].sort_values(ascending = False).index.tolist()[:610]
    # 11 bizim similarity koşulumuz gibi davranır
    for i in range(0,174):
#         display(items[items['movie id'] == mov])
        if sim[user_id][i+1] >= 0.70:
            temp_table = temp_table.append(user_df[user_df['user_id'] == sim[user_id].index[i]], ignore_index=True)
    for i in range(175,610):
        if sim[user_id][i+1] >= 0.70:
            temp_table = temp_table.append(user_df[user_df['user_id'] == sim[user_id].index[i]], ignore_index=True)
    temp_table=temp_table.sort_values(by="user_id")
    return temp_table
similar_user=rec_user1(4)
similar_user

,user_id
0,75
1,128
2,137
3,189
4,480
5,483
6,578


In [359]:
def rec_user(user_id):
    temp_table = pd.DataFrame(columns = user_df.columns)
    us = sim[user_id].sort_values(ascending = False).index.tolist()[:11]
    # 11 bizim similarity koşulumuz gibi davranır
    for u in us:
        temp_table = temp_table.append(user_df[user_df['user_id'] == u], ignore_index=True)
    return temp_table

def user_rating(x):
    similar_user = rec_user(x)
    
    similar_user = similar_user.merge(pivot_table_user , on= 'user_id')
    similar_user = similar_user.set_index('user_id')
    
    similar_user.replace(0, np.nan, inplace=True)
    u_ratings = similar_user[similar_user.index==x]
    similar_user.drop(similar_user.index[0] , inplace = True) # kendisini siler.
    u_ratings=u_ratings.append(similar_user.mean(axis = 0, skipna = True), ignore_index =True)  
    return u_ratings
user_rating(600)

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,2.5,4.0,NaN,1.5,2.5,NaN,3.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [365]:
reader = Reader(rating_scale=(1, 5))
sup_data = Dataset.load_from_df(df, reader)

In [333]:
algo = NormalPredictor()
cross_validate(algo, sup_data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm NormalPredictor on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    1.4237  1.4083  1.4048  1.4284  1.4283  1.4265  1.4346  1.4400  1.4164  1.4118  1.4223  0.0110  
MAE (testset)     1.1404  1.1216  1.1219  1.1410  1.1426  1.1412  1.1464  1.1469  1.1300  1.1304  1.1362  0.0090  
Fit time          0.12    0.14    0.14    0.13    0.13    0.15    0.15    0.13    0.13    0.14    0.14    0.01    
Test time         0.05    0.05    0.06    0.05    0.16    0.06    0.06    0.05    0.05    0.18    0.08    0.05    


{'test_rmse': array([1.42374914, 1.40825098, 1.4048277 , 1.42835457, 1.42831312,
        1.42647273, 1.43459467, 1.43999776, 1.41636517, 1.41184126]),
 'test_mae': array([1.14040658, 1.12162388, 1.12186956, 1.1409992 , 1.14257571,
        1.14115153, 1.14636906, 1.1469133 , 1.13002821, 1.13037799]),
 'fit_time': (0.11837029457092285,
  0.1360011100769043,
  0.13763189315795898,
  0.13238954544067383,
  0.13416790962219238,
  0.15264892578125,
  0.15161728858947754,
  0.1326758861541748,
  0.13199257850646973,
  0.13545727729797363),
 'test_time': (0.05085015296936035,
  0.04905056953430176,
  0.058333635330200195,
  0.05193758010864258,
  0.15904760360717773,
  0.059859275817871094,
  0.05983901023864746,
  0.048021554946899414,
  0.04801774024963379,
  0.18351387977600098)}

In [360]:
algo = SVD()
cross_validate(algo, sup_data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.8774  0.8683  0.8703  0.8660  0.8599  0.8824  0.8598  0.8657  0.8680  0.8710  0.8689  0.0067  
MAE (testset)     0.6721  0.6698  0.6682  0.6629  0.6622  0.6739  0.6580  0.6632  0.6705  0.6690  0.6670  0.0049  
Fit time          4.73    4.97    4.70    4.85    4.73    4.83    4.87    4.87    4.78    4.73    4.81    0.08    
Test time         0.07    0.07    0.06    0.06    0.05    0.06    0.05    0.05    0.05    0.05    0.06    0.01    


{'test_rmse': array([0.87735398, 0.86830516, 0.87030315, 0.86595774, 0.85990278,
        0.88236028, 0.85981011, 0.8656651 , 0.8679865 , 0.87095969]),
 'test_mae': array([0.67212885, 0.66982939, 0.66824337, 0.66285239, 0.66217516,
        0.67394736, 0.65804478, 0.66319139, 0.67047049, 0.66902427]),
 'fit_time': (4.734082937240601,
  4.970289945602417,
  4.701866149902344,
  4.846092939376831,
  4.729299545288086,
  4.830217599868774,
  4.865446329116821,
  4.8710777759552,
  4.776680946350098,
  4.7329418659210205),
 'test_time': (0.06781840324401855,
  0.06988167762756348,
  0.06201624870300293,
  0.05777764320373535,
  0.05397224426269531,
  0.0578455924987793,
  0.0520784854888916,
  0.052980661392211914,
  0.051859378814697266,
  0.053894758224487305)}

In [335]:
algo = KNNBasic(k=20)
cross_validate(algo, sup_data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.9292  0.9378  0.9329  0.9268  0.9148  0.9279  0.9299  0.9413  0.9469  0.941

{'test_rmse': array([0.92922443, 0.93778659, 0.93293045, 0.92675229, 0.91478445,
        0.92791039, 0.9299062 , 0.9413023 , 0.94694223, 0.94166932]),
 'test_mae': array([0.71280978, 0.71672164, 0.71437245, 0.70677615, 0.70157188,
        0.70851972, 0.71067729, 0.7159735 , 0.71736941, 0.71623026]),
 'fit_time': (0.21010398864746094,
  0.21541094779968262,
  0.21020102500915527,
  0.2079334259033203,
  0.21206140518188477,
  0.21044111251831055,
  0.21047091484069824,
  0.20952343940734863,
  0.25093746185302734,
  0.21715164184570312),
 'test_time': (0.6667094230651855,
  0.5808911323547363,
  0.5838818550109863,
  0.5827624797821045,
  0.5924882888793945,
  0.604576826095581,
  0.569716215133667,
  0.6116669178009033,
  0.6049909591674805,
  0.58990478515625)}

In [336]:
algo = KNNBasic(sim_options={'user_based': False} , k=20) # https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#similarity-measure-configuration
cross_validate(algo, sup_data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.9225  0.9360  0.9015  0.9095  0.9220  0.9149  0.9255  0.9225  0.9174  0.918

{'test_rmse': array([0.92252645, 0.93604437, 0.9014569 , 0.90949123, 0.9220142 ,
        0.91487563, 0.92554839, 0.92251721, 0.91737516, 0.91882718]),
 'test_mae': array([0.71250536, 0.72001215, 0.69721372, 0.70094805, 0.70897086,
        0.70857875, 0.71373238, 0.70897463, 0.71097276, 0.71118575]),
 'fit_time': (4.8770833015441895,
  4.833447456359863,
  4.765929698944092,
  4.9408979415893555,
  4.835401296615601,
  4.775308132171631,
  4.6848883628845215,
  4.686059236526489,
  4.769938945770264,
  4.844850540161133),
 'test_time': (3.6078438758850098,
  3.1006171703338623,
  3.0128252506256104,
  3.278557538986206,
  3.1437346935272217,
  3.418003559112549,
  3.185091018676758,
  3.1903274059295654,
  3.2698051929473877,
  3.4084370136260986)}

In [337]:
algo = NMF()
cross_validate(algo, sup_data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.9153  0.9146  0.9143  0.9106  0.8958  0.9140  0.9146  0.9155  0.9218  0.9121  0.9129  0.0063  
MAE (testset)     0.7016  0.7009  0.6991  0.6987  0.6892  0.6955  0.7047  0.6985  0.7065  0.6976  0.6992  0.0046  
Fit time          5.14    5.22    5.31    5.39    5.47    7.64    9.13    6.82    5.97    5.64    6.17    1.24    
Test time         0.05    0.04    0.16    0.04    0.05    0.05    0.11    0.15    0.05    0.04    0.08    0.04    


{'test_rmse': array([0.9153377 , 0.91462286, 0.91425244, 0.91060794, 0.89575098,
        0.9139598 , 0.91464031, 0.91548954, 0.92179559, 0.91210515]),
 'test_mae': array([0.70160068, 0.70090177, 0.69905311, 0.69869185, 0.68924924,
        0.69553474, 0.70473272, 0.69853618, 0.70654483, 0.69764618]),
 'fit_time': (5.144850969314575,
  5.21985650062561,
  5.312904119491577,
  5.387946844100952,
  5.466638088226318,
  7.638754367828369,
  9.131326675415039,
  6.819527864456177,
  5.974034547805786,
  5.642195463180542),
 'test_time': (0.048604488372802734,
  0.04388260841369629,
  0.1586456298828125,
  0.04490399360656738,
  0.05388236045837402,
  0.05485224723815918,
  0.11464262008666992,
  0.15361475944519043,
  0.04613780975341797,
  0.04288434982299805)}

In [361]:
sup_train = sup_data.build_full_trainset()
sup_train
algo = SVD(n_factors = 150 , lr_all = 0.005 , reg_all = 0.02 , n_epochs = 40 , init_std_dev = 0.05)
algo.fit(sup_train)
kf=KFold(n_splits=10,random_state=1)

In [362]:
def prediction_algo(uid = None , iid = None):
    predictions = []
    if uid is None:
        for ui in sup_train.all_users():
            predictions.append(algo.predict(ui, iid, verbose = False))
        return predictions
    
    if iid is None:
        for ii in sup_train.all_items():
            ii = sup_train.to_raw_iid(ii)
            predictions.append(algo.predict(uid, ii, verbose = False))
        return predictions
    return predictins.append(algo.predict(uid,iid,verbose = False))

In [366]:
predictions = prediction_algo(uid =600)
predictions.sort(key=lambda x: x.est, reverse=True)
print('#### Best Recommanded Movies are ####')
for pred in predictions[:10]:
#     print('Movie -> {} with Score-> {}'.format(sup_train.to_raw_iid(pred.iid) , pred.est))
    print('Movie -> {} with Score-> {}'.format(pred.iid , pred.est))

#### Best Recommanded Movies are ####
Movie -> 1197 with Score-> 5
Movie -> 5618 with Score-> 5
Movie -> 4993 with Score-> 4.994235123697431
Movie -> 7153 with Score-> 4.970171155946684
Movie -> 6807 with Score-> 4.908056780541329
Movie -> 1073 with Score-> 4.779554960180789
Movie -> 5952 with Score-> 4.731435042505409
Movie -> 1199 with Score-> 4.724270922278841
Movie -> 1230 with Score-> 4.7073523761648906
Movie -> 4642 with Score-> 4.704779776023149
